In [18]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
import torch
torch.cuda.device_count()
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections.abc import Sequence

from unetr import CustomedUNETR

import json
from tqdm.autonotebook import tqdm
import os

In [19]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
       AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet
from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


import torch
from monai.transforms import apply_transform
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
from tqdm import tqdm
import json
import torch

In [3]:
set_determinism(seed=1024)

In [8]:
# data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
# #json_dir = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_train_new.json'
# json_dirr = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_modified_validation_files.json'



In [20]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
#json_dir = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_train_new.json'
json_dir ='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_PET_train_new.json'
#json_dir='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_ctpt_train_new.json'

In [23]:
def datafold_read(datalist, basedir, fold=0,key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [24]:
import json
train_files, validation_files = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(train_files), len(validation_files)
# Path for the new JSON file

(39, 15)

In [25]:
import json
train_files, validation_files = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
#len(train_files), len(validation_files)
# Path for the new JSON file
#train_files, validation_files = datafold_read(datalist=json_dir_7th, basedir=data_dir, fold=0)
#ct_validation_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_ct_train_new.json'

pt_validation_json_path='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_PET_train_new'
#ctpt_validation_json_path='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_ctpt_train_new.json'
# Save the validation files to a JSON file
with open(pt_validation_json_path, 'w') as file:
    json.dump(validation_files, file, indent=4)

print(f"Validation files have been saved to {pt_validation_json_path}")

Validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_PET_train_new


In [26]:
len(validation_files)
#hgj_ct_validation_files = '/home/nada.saadi/CTPET/hecktor2022_cropped/hgj_ct_validation_files.json'

15

In [27]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [10]:
# num_samples = 4

# train_transforms = Compose(
#     [
#         LoadImaged(keys=["ct", "pt", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=["ct", "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=["ct", "pt", "seg"], axcodes="PLS"),
#         NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["ct"]),
#         ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
#         #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
#         RandCropByPosNegLabeld(
#             keys=["ctpt", "seg"],
#             label_key="seg",
#             spatial_size=(96, 96, 96),
#             pos=1,
#             neg=1,
#             num_samples=num_samples,
#             image_key="ctpt",
#             image_threshold=0,
#         ),
#         RandFlipd(
#             keys=["ctpt", "seg"],
#             spatial_axis=[0],
#             prob=0.20,
#         ),
#         RandFlipd(
#             keys=["ctpt", "seg"],
#             spatial_axis=[1],
#             prob=0.20,
#         ),
#         RandFlipd(
#             keys=["ctpt", "seg"],
#             spatial_axis=[2],
#             prob=0.20,
#         ),
#         RandRotate90d(
#             keys=["ctpt", "seg"],
#             prob=0.20,
#             max_k=3,
#         ),
#     ]
# )
# val_transforms_ctpt = Compose(
#     [
#         LoadImaged(keys=["ct", "pt", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=["ct", "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=["ct", "pt", "seg"], axcodes="PLS"),
#         NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["ct"]),
#         ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
#     ]
# )

In [28]:
num_samples = 4

train_transforms = Compose(
    [
        LoadImaged(keys=["pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["pt",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["pt",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
        #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
        RandCropByPosNegLabeld(
            keys=["pt", "seg"],
            label_key="seg",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=num_samples,
            image_key="pt",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[0],
            prob=0.20,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[1],
            prob=0.20,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[2],
            prob=0.20,
        ),
        RandRotate90d(
            keys=["pt", "seg"],
            prob=0.20,
            max_k=3,
        ),
    ]
)
val_transforms_pt = Compose(
    [
        LoadImaged(keys=["pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["pt", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["pt",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
    ]
)

In [12]:
# num_samples = 4

# train_transforms = Compose(
#     [
#         LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
#         #NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["ct"]),
#         ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
#         #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
#         RandCropByPosNegLabeld(
#             keys=["ct", "seg"],
#             label_key="seg",
#             spatial_size=(96, 96, 96),
#             pos=1,
#             neg=1,
#             num_samples=num_samples,
#             image_key="ct",
#             image_threshold=0,
#         ),
#         RandFlipd(
#             keys=["ct", "seg"],
#             spatial_axis=[0],
#             prob=0.20,
#         ),
#         RandFlipd(
#             keys=["ct", "seg"],
#             spatial_axis=[1],
#             prob=0.20,
#         ),
#         RandFlipd(
#             keys=["ct", "seg"],
#             spatial_axis=[2],
#             prob=0.20,
#         ),
#         RandRotate90d(
#             keys=["ct", "seg"],
#             prob=0.20,
#             max_k=3,
#         ),
#     ]
# )
# val_transforms = Compose(
#     [
#         LoadImaged(keys=["ct", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=["ct", "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
#         #NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["ct"]),
#         ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
#     ]
# )

In [29]:
train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=8,
    pin_memory=torch.cuda.is_available(),
    )

val_ds = monai.data.Dataset(data=validation_files, transform=val_transforms_ctpt)

val_loader = DataLoader(
    val_ds, 
    batch_size=2, 
    num_workers=8, 
    shuffle= False)

In [30]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16, #16
    hidden_size= 768, #768,
    mlp_dim=3072, #3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

model_our = CustomedUNETR(
    in_channels=1,  # Number of input channels
    out_channels=3,  # Number of output channels
    img_size=(96, 96, 96),  # Size of the input image
    feature_size=48,  # Size of the feature maps
    hidden_size=768,
    num_heads = 12,# Size of the hidden layers in the transformer
    mlp_dim=3072,  # Dimension of the MLP in the transformer
    pos_embed="perceptron",  # Type of positional embedding
    norm_name="instance",  # Type of normalization
    res_block=True,  # Whether to use residual blocks
    dropout_rate=0.0,
    proj_type= "conv",
    r=4,
    lora_layer=True,
    use_ct_encoder=True,
    use_pet_encoder=False    
).to(device) 
# pt_weights_ct_only_6th='/home/nada.saadi/CTPET/hecktor2022_cropped/6th_CT_ourapproach/6th_center_CT_our_approach.pth'
# state_dict = torch.load(pt_weights_ct_only_6th)

# for name, param in model.named_parameters():
#     if "decoder5" in name or "decoder4" in name or "decoder3" in name or "decoder2" in name:
#         param.requires_grad = False
# PT_6TH_CHANNEL_CT='/home/nada.saadi/CTPET/hecktor2022_cropped/6thCTonly_channel/6thCTonly_channel.pth'
# torch.load(PT_6TH_CHANNEL_CT)
# for name, param in model.named_parameters():
#     if "decoder5" in name or "decoder4" in name or "decoder3" in name or "decoder2" in name:
#         param.requires_grad = False

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)
/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: unetr CustomedUNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


Nada's version of UNETR
(96, 96, 96)
(16, 16, 16)
(6, 6, 6)
768
zaz w dakchi ya s lbnat


In [21]:
# val_transforms_ctpt = Compose(
#     [
#         LoadImaged(keys=["ct", "pt", "seg"], ensure_channel_first = True),
#         SpatialPadd(keys=["ct", "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
#         Orientationd(keys=["ct", "pt", "seg"], axcodes="PLS"),
#         NormalizeIntensityd(keys=["pt"]),
#         ClipCT(keys=["ct"]),
#         ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
#         #MulPTFM(keys=["ct","pt"]),
#         ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
#     ]
# )

In [15]:
import json
train_files, validation_files = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
print(len(train_files), len(validation_files))
# Path for the new JSON file
ct_validation_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json'

# Save the validation files to a JSON file
with open(ct_validation_json_path, 'w') as file:
    json.dump(validation_files, file, indent=4)

print(f"Validation files have been saved to {ct_validation_json_path}")

0 15
Validation files have been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/hmr_ct_validation_files.json


In [31]:
import io

with open(pt_validation_json_path, 'r') as f:
    test_data = json.load(f)

test_loader = DataLoader(
    dataset=Dataset(data=test_data, transform=val_transforms_pt),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
) 

In [32]:
len(test_data)

15

In [35]:
import io

# with open(json_dir, 'r') as f:
#     test_data = json.load(f)

# test_loader = DataLoader(
#     dataset=Dataset(data=test_data, transform=val_transforms),
#     batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
#     shuffle=False,
#     num_workers=4
# )

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model_our.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

rt='/home/nada.saadi/CTPET/hecktor2022_cropped/a_6th/a_6th_ctptct.pth'
model_our.load_state_dict(torch.load(rt), strict=False)
model_state_dict = model.state_dict()
state_dict = {}  # Define the "state_dict" variable
for name, param in state_dict.items():
    if name in model_state_dict:
        if param.shape == model_state_dict[name].shape:
            model_state_dict[name] = param
#model_our.encoder1_pt.load_state_dict(model_our.encoder1.state_dict(), strict=True)




def testing():
    model.eval()
    

    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["pt"].cuda(), batch_data["seg"].cuda()
            
            test_outputs_pt = sliding_window_inference(test_inputs, (96, 96, 96), 4, model_our)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs_ct = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs_pt)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs_ctpt, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs_pt, y=test_labels)
            
        # After processing all batches, save the outputs
    #ct_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_output_ct_saved.pt'
    # ct_save_path_7thcenter = '/home/nada.saadi/CTPET/hecktor2022_cropped/HMR_test_output_ct_saved.pt'
    # torch.save(test_outputs_ct, ct_save_path_7thcenter)
    # print(f"Outputs saved at: {ct_save_path_7thcenter}")
    

    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
testing()

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:02<?, ?it/s]


RuntimeError: shape '[4, 6, 6, 6, 768]' is invalid for input of size 331776

In [23]:
model_our.encoder1.state_dict().keys()

odict_keys(['layer.conv1.conv.weight', 'layer.conv2.conv.weight', 'layer.conv3.conv.weight'])

In [22]:
model_our.encoder1_pt.state_dict().keys()

odict_keys(['layer.conv1.conv.weight', 'layer.conv2.conv.weight', 'layer.conv3.conv.weight'])

In [ ]:
model_our.encoder1_pt.load_state_dict(model_our.encoder1.state_dict(), strict=False)

In [ ]:
# model_state_dict = model.state_dict()
# state_dict = {}  # Define the "state_dict" variable
# for name, param in state_dict.items():
#     if name in model_state_dict:
#         if param.shape == model_state_dict[name].shape:
#             model_state_dict[name] = param

In [ ]:
# Load the state dict from file
loaded_state_dict = torch.load(rt)

# Get the model's state dict
model_state_dict = model_our.state_dict()

# Filter out loaded state dict keys that do not match the model's state dict
filtered_state_dict = {k: v for k, v in loaded_state_dict.items() if k in model_state_dict and model_state_dict[k].size() == v.size()}

# Update the model's state dict with the filtered state dict
model_our.load_state_dict(filtered_state_dict, strict=False)

# Optionally, identify and log the keys that were ignored
ignored_keys = [k for k in loaded_state_dict if k not in filtered_state_dict]
if ignored_keys:
    print("The following keys were ignored due to mismatch:", ignored_keys)